In [2]:
import sys, os
import torch
import torch.nn as nn
sys.path.append("/home/username/open_clip_original/")
sys.path.insert(0,"/home/username/open_clip_original/src/")
from open_clip import create_model_and_transforms, trace_model, get_tokenizer, create_loss
from training.data import get_data
from training.data import get_data_transformer
from training.distributed import is_master, init_distributed_device, broadcast_object
from training.params import parse_args
from open_clip.transformer import HeadTransformer
import json
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
from glob import glob
import pandas as pd

### CLIP FT attributes cosine loss

#### VAMUS

In [ ]:
w_sum = {}
for i,f in enumerate(glob("/home/username/open_clip_original/wandb/run-202403*-vamus_clip_attr_*_3mar")):
    key = f.split("_attr_")[1].split("_3mar")[0]
    jsfile = f + "/files/wandb-summary.json"
    with open(jsfile) as jsf:
        w_sum[key] = json.load(jsf)
    # print(i,f + "/files/wandb-summary.json")

In [ ]:
w_sum['categories']

In [ ]:
attributes = []
ap = []
p1 = []
h5 = []
ar = []
r1 = []
hr5 = []
for key, summ in w_sum.items():
    attributes.append(key)
    print(key)
    ap.append(summ[f'test_{key}_mean_average_precision'] * 100)
    p1.append(summ[f'test_{key}_mean_precision_at_1'] * 100)
    h5.append(summ[f'test_{key}_mean_hit_rate_at_5'] * 100)
    ar.append(summ[f'test_{key}_mean_average_recall'] * 100)
    r1.append(summ[f'test_{key}_mean_recall_at_1'] * 100)
    hr5.append(summ[f'test_{key}_mean_hit_rate_sample_at_5'] * 100)

results_vamus = pd.DataFrame({
    "attributes":attributes,
    "AP":ap,
    "P1":p1,
    "H5":h5,
    "AR":ar,
    "R1":r1,
    "HR5":hr5,
})

results_vamus.sort_values('attributes')

categories
artistMakerPerson
materials
objectType
techniques
historicalContext
titles
placesOfOrigin
marksAndInscriptions
production
physicalDescription
materialsAndTechniques
productionDates
objectHistory
styles
briefDescription
summaryDescription
partTypes


,attributes,AP,P1,H5,AR,R1,HR5
1,artistMakerPerson,1.626435,1.772288,2.941879,6.438242,5.690000,9.985000
15,briefDescription,2.099930,1.372067,2.660603,3.217919,3.825000,8.875000
0,categories,8.337729,15.008727,25.479931,34.461445,40.125000,69.645000
5,historicalContext,0.584003,0.315021,0.782359,0.016689,0.005000,0.005000
8,marksAndInscriptions,0.571425,0.298215,0.727646,0.426475,0.005000,0.005000
2,materials,4.392290,7.923269,12.510425,37.388292,34.935001,61.659998
11,materialsAndTechniques,2.355115,1.929629,3.685973,2.671184,0.265000,3.360000
13,objectHistory,1.017619,0.732303,1.441721,1.745384,2.225000,6.020000
3,objectType,7.765743,8.048345,12.566955,1.399571,0.285000,0.465000
17,partTypes,4.945301,6.931682,10.727128,0.286339,0.120000,0.255000


#### BRITMUS

In [32]:
w_sum = {}
for i,f in enumerate(glob("/home/username/open_clip_original/wandb/run-202403*-britmus_clip_attr_*_3mar")):
    key = f.split("_attr_")[1].split("_3mar")[0]
    jsfile = f + "/files/wandb-summary.json"
    with open(jsfile) as jsf:
        w_sum[key] = json.load(jsf)
    # print(i,f + "/files/wandb-summary.json")

In [33]:
w_sum.keys()

dict_keys(['Technique', 'Materials', 'Subjects', 'Culture', 'Inscription', 'Title', 'Production place', 'Production date', 'Object type', 'Assoc name', 'Producer name', 'Curators Comments'])

In [34]:
attributes = []
ap = []
p1 = []
h5 = []
ar = []
r1 = []
hr5 = []
for key, summ in w_sum.items():
    attributes.append(key)
    print(key)
    ap.append(summ[f'test_{key}_mean_average_precision'] * 100)
    p1.append(summ[f'test_{key}_mean_precision_at_1'] * 100)
    h5.append(summ[f'test_{key}_mean_hit_rate_at_5'] * 100)
    ar.append(summ[f'test_{key}_mean_average_recall'] * 100)
    r1.append(summ[f'test_{key}_mean_recall_at_1'] * 100)
    hr5.append(summ[f'test_{key}_mean_hit_rate_sample_at_5'] * 100)

results_britmus = pd.DataFrame({
    "attributes":attributes,
    "AP":ap,
    "P1":p1,
    "H5":h5,
    "AR":ar,
    "R1":r1,
    "HR5":hr5,
})

results_britmus.sort_values('attributes')

Technique
Materials
Subjects
Culture
Inscription
Title
Production place
Production date
Object type
Assoc name
Producer name
Curators Comments


,attributes,AP,P1,H5,AR,R1,HR5
9,Assoc name,1.463615,0.884273,2.205542,0.165277,0.039038,0.132730
3,Culture,6.553227,11.244980,15.863454,64.327103,63.858527,77.498436
11,Curators Comments,0.489253,0.229208,0.610070,0.046628,0.007808,0.046846
4,Inscription,1.094590,0.484750,1.442206,0.267061,0.132730,0.249844
1,Materials,8.309048,12.834224,20.320855,76.825672,74.016237,88.304186
8,Object type,9.142371,12.076923,17.423077,0.288288,0.062461,0.156152
10,Producer name,2.380621,1.804003,3.608005,0.302366,0.031230,0.078076
7,Production date,5.128040,8.246073,14.659686,42.601296,33.252653,50.288880
6,Production place,4.678966,4.630454,8.192342,7.296153,3.138663,9.189569
2,Subjects,3.353712,5.939849,11.503759,24.817081,19.128670,39.334792


### CLIP original inference

#### VAMUS

In [31]:
clip_inf_file = "/home/username/open_clip_original/wandb/run-20240303_213511-vamus_clip_inference_3mar/files/wandb-summary.json"
with open(clip_inf_file) as f:
    clip_inf_vamus = json.load(f)

attributes = []
ap = []
p1 = []
h5 = []
ar = []
r1 = []
hr5 = []
for key in results_vamus['attributes']:
    attributes.append(key)
    print(key)
    ap.append(clip_inf_vamus[f'test_{key}_mean_average_precision'] * 100)
    p1.append(clip_inf_vamus[f'test_{key}_mean_precision_at_1'] * 100)
    h5.append(clip_inf_vamus[f'test_{key}_mean_hit_rate_at_5'] * 100)
    ar.append(clip_inf_vamus[f'test_{key}_mean_average_recall'] * 100)
    r1.append(clip_inf_vamus[f'test_{key}_mean_recall_at_1'] * 100)
    hr5.append(clip_inf_vamus[f'test_{key}_mean_hit_rate_sample_at_5'] * 100)

clip_inf_vamus = pd.DataFrame({
    "attributes":attributes,
    "AP":ap,
    "P1":p1,
    "H5":h5,
    "AR":ar,
    "R1":r1,
    "HR5":hr5,
})
clip_inf_vamus.sort_values('attributes')

categories
artistMakerPerson
materials
objectType
techniques
historicalContext
titles
placesOfOrigin
marksAndInscriptions
production
physicalDescription
materialsAndTechniques
productionDates
objectHistory
styles
briefDescription
summaryDescription
partTypes


,attributes,AP,P1,H5,AR,R1,HR5
1,artistMakerPerson,0.328595,0.262561,0.584795,0.128011,0.010,0.025000
15,briefDescription,1.989991,1.315864,2.485808,3.419777,5.050,9.745000
0,categories,5.706804,9.773124,21.291448,11.962684,8.710,25.584999
5,historicalContext,0.724321,0.353100,0.903521,0.084024,0.015,0.020000
8,marksAndInscriptions,0.717074,0.493845,0.835003,0.296295,0.210,0.415000
2,materials,1.420671,2.251877,4.670559,8.590925,3.190,13.435000
11,materialsAndTechniques,0.705876,0.381814,0.945724,0.815862,0.130,0.670000
13,objectHistory,0.590905,0.381408,0.826383,0.103394,0.060,0.130000
3,objectType,3.927074,3.516001,6.784783,2.158308,0.310,0.995000
17,partTypes,3.081383,3.715541,6.871754,0.583687,0.075,0.240000


#### BRITMUS

In [36]:
clip_inf_file = "/home/username/open_clip_original/wandb/run-20240303_221331-britmus_clip_inference_3mar/files/wandb-summary.json"
with open(clip_inf_file) as f:
    clip_inf_brit = json.load(f)
# clip_inf_brit

# clip_inf_file = "/home/username/open_clip_original/wandb/run-20240303_213511-vamus_clip_inference_3mar/files/wandb-summary.json"
# with open(clip_inf_file) as f:
#     clip_inf_vamus = json.load(f)

attributes = []
ap = []
p1 = []
h5 = []
ar = []
r1 = []
hr5 = []
for key in results_britmus['attributes']:
    if key == 'Title': continue
    attributes.append(key)
    print(key)
    ap.append(clip_inf_brit[f'test_{key}_mean_average_precision'] * 100)
    p1.append(clip_inf_brit[f'test_{key}_mean_precision_at_1'] * 100)
    h5.append(clip_inf_brit[f'test_{key}_mean_hit_rate_at_5'] * 100)
    ar.append(clip_inf_brit[f'test_{key}_mean_average_recall'] * 100)
    r1.append(clip_inf_brit[f'test_{key}_mean_recall_at_1'] * 100)
    hr5.append(clip_inf_brit[f'test_{key}_mean_hit_rate_sample_at_5'] * 100)

clip_inf_britmus = pd.DataFrame({
    "attributes":attributes,
    "AP":ap,
    "P1":p1,
    "H5":h5,
    "AR":ar,
    "R1":r1,
    "HR5":hr5,
})
clip_inf_britmus.sort_values('attributes')

Technique
Materials
Subjects
Culture
Inscription
Production place
Production date
Object type
Assoc name
Producer name
Curators Comments


,attributes,AP,P1,H5,AR,R1,HR5
8,Assoc name,0.648670,0.395867,0.838003,0.082773,0.007808,0.054653
3,Culture,0.982671,0.803213,3.212851,3.471356,1.460025,5.270144
10,Curators Comments,0.377376,0.193728,0.437065,0.120095,0.054653,0.242036
4,Inscription,0.255379,0.117424,0.286033,0.077031,0.023423,0.070269
1,Materials,1.554790,2.317291,6.417112,16.163202,8.713304,25.070268
7,Object type,3.405169,4.038462,8.000000,0.404504,0.078076,0.171768
9,Producer name,0.666195,0.357042,0.873814,0.090280,0.023423,0.062461
6,Production date,0.185026,0.000000,0.523560,1.218160,0.398189,1.210181
5,Production place,0.589248,0.463045,0.943900,0.479865,0.117114,0.343535
2,Subjects,1.806100,3.233083,6.616541,2.833367,1.584947,4.122423


### CLIP FT Captions

#### VAMUS

In [47]:

clip_inf_file = "/home/username/open_clip_original/wandb/run-20240304_121258-vamus_clip_f_inference_4mar/files/wandb-summary.json"
with open(clip_inf_file) as f:
    clip_inf_vamus = json.load(f)

attributes = []
ap = []
p1 = []
h5 = []
ar = []
r1 = []
hr5 = []
for key in results_vamus['attributes']:
    attributes.append(key)
    print(key)
    ap.append(clip_inf_vamus[f'test_{key}_mean_average_precision'] * 100)
    p1.append(clip_inf_vamus[f'test_{key}_mean_precision_at_1'] * 100)
    h5.append(clip_inf_vamus[f'test_{key}_mean_hit_rate_at_5'] * 100)
    ar.append(clip_inf_vamus[f'test_{key}_mean_average_recall'] * 100)
    r1.append(clip_inf_vamus[f'test_{key}_mean_recall_at_1'] * 100)
    hr5.append(clip_inf_vamus[f'test_{key}_mean_hit_rate_sample_at_5'] * 100)

clip_inf_vamus = pd.DataFrame({
    "attributes":attributes,
    "AP":ap,
    "P1":p1,
    "H5":h5,
    "AR":ar,
    "R1":r1,
    "HR5":hr5,
})
clip_inf_vamus.sort_values('attributes')


categories
artistMakerPerson
summaryDescription
objectType
historicalContext
materials
titles
marksAndInscriptions
placesOfOrigin
physicalDescription
materialsAndTechniques
production
productionDates
objectHistory
briefDescription
styles
partTypes
techniques


,attributes,AP,P1,H5,AR,R1,HR5
1,artistMakerPerson,0.333579,0.304332,0.519155,0.099864,0.010,0.020000
14,briefDescription,1.929545,1.222537,2.438371,2.879062,3.505,7.685000
0,categories,6.089076,10.645724,20.244329,15.840700,12.750,36.175001
4,historicalContext,0.953131,0.484647,1.246235,0.053917,0.010,0.015000
7,marksAndInscriptions,0.642355,0.427045,0.772975,0.127148,0.075,0.160000
5,materials,1.564066,2.251877,5.004170,9.112486,4.430,14.910001
10,materialsAndTechniques,0.848843,0.455240,1.210056,0.743443,0.125,0.435000
13,objectHistory,0.672005,0.417006,0.902665,0.085634,0.035,0.070000
3,objectType,4.420229,3.914297,7.334157,1.427300,0.190,0.500000
16,partTypes,3.345759,4.015182,7.151418,0.331253,0.045,0.120000


#### BRITMUS

In [49]:
clip_inf_file = "/home/username/open_clip_original/wandb/run-20240304_125230-britmus_clip_f_inference_4mar/files/wandb-summary.json"
with open(clip_inf_file) as f:
    clip_inf_brit = json.load(f)
# clip_inf_brit

# clip_inf_file = "/home/username/open_clip_original/wandb/run-20240303_213511-vamus_clip_inference_3mar/files/wandb-summary.json"
# with open(clip_inf_file) as f:
#     clip_inf_vamus = json.load(f)

attributes = []
ap = []
p1 = []
h5 = []
ar = []
r1 = []
hr5 = []
for key in results_britmus['attributes']:
    # if key == 'Title': continue
    attributes.append(key)
    print(key)
    ap.append(clip_inf_brit[f'test_{key}_mean_average_precision'] * 100)
    p1.append(clip_inf_brit[f'test_{key}_mean_precision_at_1'] * 100)
    h5.append(clip_inf_brit[f'test_{key}_mean_hit_rate_at_5'] * 100)
    ar.append(clip_inf_brit[f'test_{key}_mean_average_recall'] * 100)
    r1.append(clip_inf_brit[f'test_{key}_mean_recall_at_1'] * 100)
    hr5.append(clip_inf_brit[f'test_{key}_mean_hit_rate_sample_at_5'] * 100)

clip_inf_britmus = pd.DataFrame({
    "attributes":attributes,
    "AP":ap,
    "P1":p1,
    "H5":h5,
    "AR":ar,
    "R1":r1,
    "HR5":hr5,
})
clip_inf_britmus.sort_values('attributes')

Technique
Materials
Subjects
Culture
Inscription
Title
Production place
Production date
Object type
Assoc name
Producer name
Curators Comments


,attributes,AP,P1,H5,AR,R1,HR5
9,Assoc name,0.267353,0.113105,0.323891,0.079576,0.015615,0.031230
3,Culture,1.316209,3.413654,6.224900,7.570811,5.176452,13.780449
11,Curators Comments,0.296238,0.127477,0.357941,0.077133,0.015615,0.109307
4,Inscription,0.231328,0.063228,0.231837,0.083918,0.015615,0.085884
1,Materials,1.695756,3.386809,6.417112,18.944322,11.383510,28.263584
8,Object type,2.183518,3.307692,6.307692,0.268022,0.023423,0.109307
10,Producer name,0.551445,0.319459,0.836230,0.098601,0.015615,0.070269
7,Production date,0.184306,0.130890,0.654450,0.876133,0.218613,0.687071
6,Production place,0.413262,0.320570,0.801425,0.498276,0.070269,0.273267
2,Subjects,0.790504,1.804511,3.834587,2.937153,1.881636,5.145222


### MUSE over CLIP original

In [55]:
import shutil

In [62]:
all_keys = []
for f in glob("/home/username/open_clip/saved_models/final_keys/*"):
    all_keys.append(f.split('/')[-1])
print(all_keys)
for key in all_keys:
    best_path = None
    best_perf = 0
    for f in glob(f'/home/username/open_clip/saved_models/final_keys/{key}/2_T_F_head/*'):
        perf = float(f.split('_')[-1][:-3])
        if perf > best_perf:
            best_perf = perf
            best_path = f
    # print(key,best_perf,best_path)
    # print(os.path.dirname(best_path) + '/model.pt')
    shutil.copy2(best_path,os.path.dirname(best_path) + '/model.pt')

['Technique', 'categories', 'artistMakerPerson', 'Materials', 'objectType', 'historicalContext', 'Subjects', 'placesOfOrigin', 'marksAndInscriptions', 'Culture', 'production', 'Inscription', 'materialsAndTechniques', 'productionDates', 'physicalDescription', 'Title', 'objectHistory', 'styles', 'Production place', 'partTypes', 'Production date', 'techniques', 'briefDescription', 'Object type', 'summaryDescription', 'materials', 'Assoc name', 'Producer name', 'titles', 'Curators Comments']


#### VAMUS

In [79]:
from genericpath import isfile


w_sum = {}
for i,f in enumerate(glob("/home/username/open_clip_original/wandb/run-2024*-vamus_*_T_F_inference2")):
    key = f.split("-vamus_")[1].split("_T")[0]
    jsfile = f + "/files/wandb-summary.json"
    if os.path.isfile(jsfile):
        with open(jsfile) as jsf:
            w_sum[key] = json.load(jsf)
    # print(i,f + "/files/wandb-summary.json")

In [80]:
w_sum.keys()

dict_keys(['categories', 'artistMakerPerson', 'historicalContext', 'marksAndInscriptions', 'materials', 'materialsAndTechniques', 'objectHistory', 'objectType', 'partTypes', 'physicalDescription', 'placesOfOrigin', 'production', 'productionDates', 'styles', 'summaryDescription', 'techniques', 'briefDescription', 'titles'])

In [81]:
attributes = []
ap = []
p1 = []
h5 = []
ar = []
r1 = []
hr5 = []
for key, summ in w_sum.items():
    if 'test_mean_average_precision' not in summ: continue
    attributes.append(key)
    print(key)
    ap.append(summ[f'test_mean_average_precision'] * 100)
    p1.append(summ[f'test_mean_precision_at_1'] * 100)
    h5.append(summ[f'test_mean_hit_rate_at_5'] * 100)
    ar.append(summ[f'test_mean_average_recall'] * 100)
    r1.append(summ[f'test_mean_recall_at_1'] * 100)
    hr5.append(summ[f'test_mean_hit_rate_sample_at_5'] * 100)

results_vamus = pd.DataFrame({
    "attributes":attributes,
    "AP":ap,
    "P1":p1,
    "H5":h5,
    "AR":ar,
    "R1":r1,
    "HR5":hr5,
})

results_vamus.sort_values('attributes')

categories
artistMakerPerson
historicalContext
marksAndInscriptions
materials
materialsAndTechniques
objectHistory
objectType
partTypes
physicalDescription
placesOfOrigin
production
productionDates
styles
summaryDescription
techniques
briefDescription
titles


,attributes,AP,P1,H5,AR,R1,HR5
1,artistMakerPerson,1.049483,0.936866,1.527629,0.260488,0.000000,0.045000
16,briefDescription,2.463393,1.673705,3.088568,0.978536,0.645000,1.515000
0,categories,11.815265,17.975567,23.036650,57.665622,73.439997,80.925000
2,historicalContext,0.477275,0.231938,0.508879,0.468534,0.000000,0.005000
3,marksAndInscriptions,0.545599,0.236187,0.639374,0.609454,0.000000,0.060000
4,materials,6.166775,9.007506,12.510425,76.360238,84.600002,90.539998
5,materialsAndTechniques,3.163386,2.208647,4.384986,7.413502,3.870000,6.470000
6,objectHistory,0.586211,0.350895,0.611524,0.778871,0.330000,0.625000
7,objectType,17.380069,16.494986,21.178409,6.497592,1.145000,7.515000
8,partTypes,13.433924,15.381542,18.058330,0.821013,0.340000,0.740000


#### BRITMUS

In [69]:
w_sum = {}
for i,f in enumerate(glob("/home/username/open_clip_original/wandb/run-2024*-britmus_*_T_F_inference2")):
    key = f.split("-britmus_")[1].split("_T")[0]
    jsfile = f + "/files/wandb-summary.json"
    with open(jsfile) as jsf:
        w_sum[key] = json.load(jsf)
    # print(i,f + "/files/wandb-summary.json")

In [70]:
w_sum.keys()

dict_keys(['Technique', 'Materials', 'Subjects', 'Culture', 'Inscription', 'Title', 'Production place', 'Production date', 'Object type', 'Assoc name', 'Producer name', 'Curators Comments'])

In [71]:
attributes = []
ap = []
p1 = []
h5 = []
ar = []
r1 = []
hr5 = []
for key, summ in w_sum.items():
    if 'test_mean_average_precision' not in summ: continue
    attributes.append(key)
    print(key)
    ap.append(summ[f'test_mean_average_precision'] * 100)
    p1.append(summ[f'test_mean_precision_at_1'] * 100)
    h5.append(summ[f'test_mean_hit_rate_at_5'] * 100)
    ar.append(summ[f'test_mean_average_recall'] * 100)
    r1.append(summ[f'test_mean_recall_at_1'] * 100)
    hr5.append(summ[f'test_mean_hit_rate_sample_at_5'] * 100)

results_britmus = pd.DataFrame({
    "attributes":attributes,
    "AP":ap,
    "P1":p1,
    "H5":h5,
    "AR":ar,
    "R1":r1,
    "HR5":hr5,
})

results_britmus.sort_values('attributes')

Technique
Materials
Subjects
Culture
Inscription
Title
Production place
Production date
Object type
Assoc name
Producer name
Curators Comments


,attributes,AP,P1,H5,AR,R1,HR5
9,Assoc name,1.020191,0.498689,1.259575,2.938794,0.023423,0.039038
3,Culture,6.123799,8.032128,11.244980,73.513591,81.480324,86.203933
11,Curators Comments,0.163996,0.055261,0.163585,0.228179,0.000000,0.007808
4,Inscription,0.383856,0.118929,0.371842,0.549303,0.007808,0.023423
1,Materials,5.616037,7.486631,11.586453,77.450782,78.466582,87.179887
8,Object type,3.460580,4.576923,7.192308,0.748595,0.359151,0.733916
10,Producer name,1.361195,0.695293,1.728836,0.268626,0.007808,0.046846
7,Production date,2.558573,3.141361,7.198953,45.180458,32.339162,58.190191
6,Production place,2.375942,1.869991,3.704363,38.017118,33.010617,40.310743
2,Subjects,1.870149,2.857143,4.586466,52.375036,59.353530,63.733602


### MUSE over CLIP F

#### VAMUS

In [39]:
w_sum = {}
for i,f in enumerate(glob("/home/username/open_clip_original/wandb/run-2024*-vamus_*_T_CLIP_F")):
    key = f.split("-vamus_")[1].split("_T")[0]
    jsfile = f + "/files/wandb-summary.json"
    with open(jsfile) as jsf:
        w_sum[key] = json.load(jsf)
    # print(i,f + "/files/wandb-summary.json")

In [40]:
w_sum.keys()

dict_keys(['categories', 'artistMakerPerson', 'summaryDescription', 'objectType', 'historicalContext', 'materials', 'titles', 'marksAndInscriptions', 'placesOfOrigin', 'physicalDescription', 'materialsAndTechniques', 'production', 'productionDates', 'objectHistory', 'briefDescription', 'styles', 'partTypes', 'techniques'])

In [42]:
attributes = []
ap = []
p1 = []
h5 = []
ar = []
r1 = []
hr5 = []
for key, summ in w_sum.items():
    attributes.append(key)
    print(key)
    ap.append(summ[f'test_mean_average_precision'] * 100)
    p1.append(summ[f'test_mean_precision_at_1'] * 100)
    h5.append(summ[f'test_mean_hit_rate_at_5'] * 100)
    ar.append(summ[f'test_mean_average_recall'] * 100)
    r1.append(summ[f'test_mean_recall_at_1'] * 100)
    hr5.append(summ[f'test_mean_hit_rate_sample_at_5'] * 100)

results_vamus = pd.DataFrame({
    "attributes":attributes,
    "AP":ap,
    "P1":p1,
    "H5":h5,
    "AR":ar,
    "R1":r1,
    "HR5":hr5,
})

results_vamus.sort_values('attributes')

categories
artistMakerPerson
summaryDescription
objectType
historicalContext
materials
titles
marksAndInscriptions
placesOfOrigin
physicalDescription
materialsAndTechniques
production
productionDates
objectHistory
briefDescription
styles
partTypes
techniques


,attributes,AP,P1,H5,AR,R1,HR5
1,artistMakerPerson,1.010406,0.901062,1.491825,0.250603,0.005000,0.035000
14,briefDescription,2.441178,1.653596,3.072584,1.083279,0.805000,1.615000
0,categories,12.265591,19.546248,23.560210,58.432877,74.945003,81.954998
4,historicalContext,0.629095,0.346176,0.688891,0.468663,0.005000,0.005000
7,marksAndInscriptions,0.522595,0.219487,0.596431,0.627777,0.020000,0.080000
5,materials,6.011603,8.507089,12.760633,76.249212,84.460002,90.530002
10,materialsAndTechniques,3.153037,2.223332,4.402608,7.438944,3.840000,6.590000
13,objectHistory,0.633697,0.395393,0.700519,0.737102,0.290000,0.560000
3,objectType,17.210415,16.604862,20.711441,6.319387,1.060000,7.300000
16,partTypes,13.568446,15.341590,18.537755,0.845134,0.360000,0.750000


#### BRITMUS

In [44]:
w_sum = {}
for i,f in enumerate(glob("/home/username/open_clip_original/wandb/run-2024*-britmus_*_T_CLIP_F")):
    key = f.split("-britmus_")[1].split("_T")[0]
    jsfile = f + "/files/wandb-summary.json"
    with open(jsfile) as jsf:
        w_sum[key] = json.load(jsf)
    # print(i,f + "/files/wandb-summary.json")

In [45]:
w_sum.keys()

dict_keys(['Technique', 'Materials', 'Subjects', 'Culture', 'Inscription', 'Title', 'Production place', 'Production date', 'Object type', 'Assoc name', 'Producer name', 'Curators Comments'])

In [46]:
attributes = []
ap = []
p1 = []
h5 = []
ar = []
r1 = []
hr5 = []
for key, summ in w_sum.items():
    attributes.append(key)
    print(key)
    ap.append(summ[f'test_mean_average_precision'] * 100)
    p1.append(summ[f'test_mean_precision_at_1'] * 100)
    h5.append(summ[f'test_mean_hit_rate_at_5'] * 100)
    ar.append(summ[f'test_mean_average_recall'] * 100)
    r1.append(summ[f'test_mean_recall_at_1'] * 100)
    hr5.append(summ[f'test_mean_hit_rate_sample_at_5'] * 100)

results_britmus = pd.DataFrame({
    "attributes":attributes,
    "AP":ap,
    "P1":p1,
    "H5":h5,
    "AR":ar,
    "R1":r1,
    "HR5":hr5,
})

results_britmus.sort_values('attributes')

Technique
Materials
Subjects
Culture
Inscription
Title
Production place
Production date
Object type
Assoc name
Producer name
Curators Comments


,attributes,AP,P1,H5,AR,R1,HR5
9,Assoc name,1.019250,0.442137,1.362398,2.853398,0.023423,0.054653
3,Culture,7.295148,9.638554,13.052209,73.682511,81.730169,86.430359
11,Curators Comments,0.155047,0.045842,0.147572,0.239256,0.000000,0.000000
4,Inscription,0.433616,0.140005,0.468190,0.549048,0.007808,0.054653
1,Materials,5.241434,6.773619,9.447415,78.157502,79.676765,87.406307
8,Object type,2.970036,3.769231,6.076923,0.765433,0.390381,0.788570
10,Producer name,1.418622,0.798647,1.766419,0.269722,0.000000,0.062461
7,Production date,2.412037,3.534031,6.151833,45.968002,33.525920,58.268267
6,Production place,2.252602,1.745325,3.241318,37.962094,32.721737,40.552780
2,Subjects,1.605722,2.781955,4.360902,51.761907,58.166772,63.077766


### MUSE over CLIP Fatt

#### VAMUS

In [72]:
w_sum = {}
for i,f in enumerate(glob("/home/username/open_clip_original/wandb/run-2024*-vamus_*_T_CLIP_Fatt")):
    key = f.split("-vamus_")[1].split("_T")[0]
    jsfile = f + "/files/wandb-summary.json"
    with open(jsfile) as jsf:
        w_sum[key] = json.load(jsf)
    # print(i,f + "/files/wandb-summary.json")

In [73]:
w_sum.keys()

dict_keys(['categories', 'artistMakerPerson', 'summaryDescription', 'objectType', 'historicalContext', 'titles', 'materials', 'placesOfOrigin', 'physicalDescription', 'marksAndInscriptions', 'production', 'materialsAndTechniques', 'productionDates', 'briefDescription', 'objectHistory', 'styles', 'techniques', 'partTypes'])

In [74]:
attributes = []
ap = []
p1 = []
h5 = []
ar = []
r1 = []
hr5 = []
for key, summ in w_sum.items():
    if 'test_mean_average_precision' not in summ: continue
    attributes.append(key)
    print(key)
    ap.append(summ[f'test_mean_average_precision'] * 100)
    p1.append(summ[f'test_mean_precision_at_1'] * 100)
    h5.append(summ[f'test_mean_hit_rate_at_5'] * 100)
    ar.append(summ[f'test_mean_average_recall'] * 100)
    r1.append(summ[f'test_mean_recall_at_1'] * 100)
    hr5.append(summ[f'test_mean_hit_rate_sample_at_5'] * 100)

results_vamus = pd.DataFrame({
    "attributes":attributes,
    "AP":ap,
    "P1":p1,
    "H5":h5,
    "AR":ar,
    "R1":r1,
    "HR5":hr5,
})

results_vamus.sort_values('attributes')

categories
artistMakerPerson
summaryDescription
objectType
historicalContext
titles
materials
placesOfOrigin
physicalDescription
marksAndInscriptions
production
materialsAndTechniques
productionDates
briefDescription
objectHistory
styles
techniques
partTypes


,attributes,AP,P1,H5,AR,R1,HR5
1,artistMakerPerson,1.178332,1.253133,2.052751,0.288297,0.035000,0.070000
13,briefDescription,2.500782,1.708767,3.147349,1.084002,0.775000,1.595000
0,categories,12.173788,18.848167,24.083769,58.750129,75.150001,82.515001
4,historicalContext,0.566893,0.308097,0.612732,0.468651,0.000000,0.005000
9,marksAndInscriptions,0.749112,0.379330,0.987690,0.617172,0.015000,0.075000
6,materials,5.954251,7.923269,12.343620,76.277500,84.364998,90.675002
11,materialsAndTechniques,2.620678,1.982495,3.912124,7.165313,3.955000,6.180000
14,objectHistory,0.754204,0.600081,1.055228,0.554937,0.110000,0.325000
3,objectType,13.283944,12.580690,17.305315,5.718115,0.970000,6.365000
17,partTypes,12.337021,13.963243,17.558929,0.887000,0.355000,0.830000


#### BRITMUS

In [3]:
w_sum = {}
for i,f in enumerate(glob("/home/username/open_clip_original/wandb/run-2024*-britmus_*_T_CLIP_Fatt")):
    key = f.split("-britmus_")[1].split("_T")[0]
    jsfile = f + "/files/wandb-summary.json"
    if os.path.isfile(jsfile):
        with open(jsfile) as jsf:
            w_sum[key] = json.load(jsf)
    # print(i,f + "/files/wandb-summary.json")

In [4]:
w_sum.keys()

dict_keys(['Technique', 'Materials', 'Subjects', 'Culture', 'Inscription', 'Title', 'Production place', 'Production date', 'Object type', 'Assoc name', 'Producer name', 'Curators Comments'])

In [5]:
attributes = []
ap = []
p1 = []
h5 = []
ar = []
r1 = []
hr5 = []
for key, summ in w_sum.items():
    if 'test_mean_average_precision' not in summ: continue
    attributes.append(key)
    print(key)
    ap.append(summ[f'test_mean_average_precision'] * 100)
    p1.append(summ[f'test_mean_precision_at_1'] * 100)
    h5.append(summ[f'test_mean_hit_rate_at_5'] * 100)
    ar.append(summ[f'test_mean_average_recall'] * 100)
    r1.append(summ[f'test_mean_recall_at_1'] * 100)
    hr5.append(summ[f'test_mean_hit_rate_sample_at_5'] * 100)

results_britmus = pd.DataFrame({
    "attributes":attributes,
    "AP":ap,
    "P1":p1,
    "H5":h5,
    "AR":ar,
    "R1":r1,
    "HR5":hr5,
})

results_britmus.sort_values('attributes')

Technique
Materials
Subjects
Culture
Inscription
Title
Production place
Production date
Object type
Assoc name
Producer name
Curators Comments


,attributes,AP,P1,H5,AR,R1,HR5
9,Assoc name,1.236790,0.632358,1.758264,3.033624,0.023423,0.062461
3,Culture,7.494403,10.040161,14.457831,73.778260,81.371015,86.313242
11,Curators Comments,0.177852,0.062483,0.173633,0.245884,0.000000,0.000000
4,Inscription,0.446831,0.170114,0.495288,0.665209,0.023423,0.062461
1,Materials,6.841376,10.695187,13.903743,79.380655,80.957216,88.421297
8,Object type,5.737629,7.807692,11.846154,0.826618,0.351343,0.882261
10,Producer name,1.664034,1.155689,2.433524,0.273617,0.007808,0.054653
7,Production date,2.808660,5.104712,9.685864,48.741537,40.302935,56.722361
6,Production place,2.755928,3.009795,5.271594,38.158295,32.823235,41.114929
2,Subjects,1.898634,3.984962,5.413534,52.684462,59.611183,64.326984
